Analyze web-hosted JSON data
============================

This notebook reads and processes JSON-encoded data hosted on the web using a combination of [Dask Bag](https://docs.dask.org/en/latest/bag.html) and [Dask Dataframe](https://docs.dask.org/en/latest/dataframe.html).

This data comes from [mybinder.org](https://mybinder.org) a web service to run Jupyter notebooks live on the web (you may be running this notebook there now).  My Binder publishes records for every time someone launches a live notebook like this one, and stores that record in a publicly accessible JSON file, one file per day. 

## Introduction to the dataset

This data is stored as JSON-encoded text files on the public web.  Here are some example lines.

In [1]:
import dask.bag as db
db.read_text('https://archive.analytics.mybinder.org/events-2018-11-03.jsonl').take(3)

('{"timestamp": "2018-11-03T00:00:00+00:00", "schema": "binderhub.jupyter.org/launch", "version": 1, "provider": "GitHub", "spec": "Qiskit/qiskit-tutorial/master", "status": "success"}\n',
 '{"timestamp": "2018-11-03T00:00:00+00:00", "schema": "binderhub.jupyter.org/launch", "version": 1, "provider": "GitHub", "spec": "ipython/ipython-in-depth/master", "status": "success"}\n',
 '{"timestamp": "2018-11-03T00:00:00+00:00", "schema": "binderhub.jupyter.org/launch", "version": 1, "provider": "GitHub", "spec": "QISKit/qiskit-tutorial/master", "status": "success"}\n')

We see that it includes one line for every time someone started a live notebook on the site.  It includes the time that the notebook was started, as well as the repository from which it was served.

In this notebook we'll look at many such files, parse them from JSON to Python dictionaries, and then from there to Pandas dataframes.  We'll then do some simple analyses on this data.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will start the dashboard which 
is useful to gain insight on the computation.  

In [2]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=1, 
                n_workers=4,
                memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:38615 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.00 GB


## Get a list of files on the web

The mybinder.org team maintains an index file that points to all other available JSON files of data.  Lets convert this to a list of URLs that we'll read in the next section.

In [3]:
import dask.bag as db
import json

In [4]:
db.read_text('https://archive.analytics.mybinder.org/index.jsonl').map(json.loads).compute()

[{'name': 'events-2018-11-03.jsonl', 'date': '2018-11-03', 'count': '7057'},
 {'name': 'events-2018-11-04.jsonl', 'date': '2018-11-04', 'count': '7489'},
 {'name': 'events-2018-11-05.jsonl', 'date': '2018-11-05', 'count': '13590'},
 {'name': 'events-2018-11-06.jsonl', 'date': '2018-11-06', 'count': '13920'},
 {'name': 'events-2018-11-07.jsonl', 'date': '2018-11-07', 'count': '12766'},
 {'name': 'events-2018-11-08.jsonl', 'date': '2018-11-08', 'count': '14105'},
 {'name': 'events-2018-11-09.jsonl', 'date': '2018-11-09', 'count': '11843'},
 {'name': 'events-2018-11-10.jsonl', 'date': '2018-11-10', 'count': '7047'},
 {'name': 'events-2018-11-11.jsonl', 'date': '2018-11-11', 'count': '6940'},
 {'name': 'events-2018-11-12.jsonl', 'date': '2018-11-12', 'count': '16322'},
 {'name': 'events-2018-11-13.jsonl', 'date': '2018-11-13', 'count': '16530'},
 {'name': 'events-2018-11-14.jsonl', 'date': '2018-11-14', 'count': '14099'},
 {'name': 'events-2018-11-15.jsonl', 'date': '2018-11-15', 'count': 

In [5]:
filenames = (db.read_text('https://archive.analytics.mybinder.org/index.jsonl')
               .map(json.loads)
               .pluck('name')
               .compute())

filenames = ['https://archive.analytics.mybinder.org/' + fn for fn in filenames]
filenames[:5]

['https://archive.analytics.mybinder.org/events-2018-11-03.jsonl',
 'https://archive.analytics.mybinder.org/events-2018-11-04.jsonl',
 'https://archive.analytics.mybinder.org/events-2018-11-05.jsonl',
 'https://archive.analytics.mybinder.org/events-2018-11-06.jsonl',
 'https://archive.analytics.mybinder.org/events-2018-11-07.jsonl']

## Create Bag of all events

We now create a [Dask Bag](https://docs.dask.org/en/latest/bag.html) around that list of URLs, and then call the `json.loads` function on every line to turn those lines of JSON-encoded text into Python dictionaries that can be more easily manipulated.

In [6]:
events = db.read_text(filenames).map(json.loads)
events.take(2)

({'timestamp': '2018-11-03T00:00:00+00:00',
  'schema': 'binderhub.jupyter.org/launch',
  'version': 1,
  'provider': 'GitHub',
  'spec': 'Qiskit/qiskit-tutorial/master',
  'status': 'success'},
 {'timestamp': '2018-11-03T00:00:00+00:00',
  'schema': 'binderhub.jupyter.org/launch',
  'version': 1,
  'provider': 'GitHub',
  'spec': 'ipython/ipython-in-depth/master',
  'status': 'success'})

## Most Popular Binders

Lets do a simple frequency count to find those binders that are run the most often.

In [7]:
events.pluck('spec').frequencies(sort=True).take(20)

(('ipython/ipython-in-depth/master', 3201358),
 ('jupyterlab/jupyterlab-demo/try.jupyter.org', 1088503),
 ('jupyterlab/jupyterlab-demo/master', 356594),
 ('ines/spacy-io-binder/live', 303942),
 ('binder-examples/requirements/master', 281650),
 ('DS-100/textbook/master', 214885),
 ('bokeh/bokeh-notebooks/master', 134552),
 ('binder-examples/r/master', 118138),
 ('QuantStack/xeus-cling/stable', 78875),
 ('ines/spacy-course/binder', 67951),
 ('ELC/8fdc0f490b3058872a7014f01416dfb6/master', 64773),
 ('rationalmatter/juno-demo-notebooks/master', 64570),
 ('Petlja/AnalizaPodatakaGim2/master', 50495),
 ('rasahq/docs-binder/master', 48531),
 ('numba/numba-examples/master', 46507),
 ('binder-examples/demo-julia/master', 32601),
 ('dask/dask-examples/master', 32171),
 ('SamLau95/nbinteract/master', 27783),
 ('wshuyi/demo-spacy-text-processing/master', 26167),
 ('ELC/380e584b87227b15727ec886223d9d4a/master', 25221))

## Convert to Dask Dataframe

Finally, we can convert our bag of Python dictionaries into a [Dask Dataframe](https://docs.dask.org/en/latest/dataframe.html), and follow up with more Pandas-like computations.

We'll do the same computation as above, now with Pandas syntax.

In [8]:
df = events.to_dataframe()
df.head()

,timestamp,schema,version,provider,spec,status
0,2018-11-03T00:00:00+00:00,binderhub.jupyter.org/launch,1,GitHub,Qiskit/qiskit-tutorial/master,success
1,2018-11-03T00:00:00+00:00,binderhub.jupyter.org/launch,1,GitHub,ipython/ipython-in-depth/master,success
2,2018-11-03T00:00:00+00:00,binderhub.jupyter.org/launch,1,GitHub,QISKit/qiskit-tutorial/master,success
3,2018-11-03T00:01:00+00:00,binderhub.jupyter.org/launch,1,GitHub,QISKit/qiskit-tutorial/master,success
4,2018-11-03T00:01:00+00:00,binderhub.jupyter.org/launch,1,GitHub,jupyterlab/jupyterlab-demo/master,success


In [9]:
df.spec.value_counts().nlargest(20).to_frame().compute()

,spec
ipython/ipython-in-depth/master,3201358
jupyterlab/jupyterlab-demo/try.jupyter.org,1088503
jupyterlab/jupyterlab-demo/master,356594
ines/spacy-io-binder/live,303942
binder-examples/requirements/master,281650
DS-100/textbook/master,214885
bokeh/bokeh-notebooks/master,134552
binder-examples/r/master,118138
QuantStack/xeus-cling/stable,78875
ines/spacy-course/binder,67951


## Persist in memory

This dataset fits nicely into memory. Lets avoid downloading data every time we do an operation and instead keep the data local in memory.

In [10]:
df = df.persist()

Honestly, at this point it makes more sense to just switch to Pandas, but this is a Dask example, so we'll continue with Dask dataframe.

## Investigate providers other than Github

Most binders are specified as git repositories on GitHub, but not all.  Lets investigate other providers.

In [11]:
import urllib

In [12]:
df.provider.value_counts().compute()

GitHub        8271263
Gist           101351
Git             31915
GitLab          31856
Zenodo            846
Hydroshare        305
Figshare          172
Dataverse          86
Name: provider, dtype: int64

In [13]:
(df[df.provider == 'GitLab']
 .spec
 .map(urllib.parse.unquote, meta=('spec', object))
 .value_counts()
 .to_frame()
 .compute())

,spec
DGothrek/ipyaggrid/binder-demo,1737
lfortran/web/lfortran-binder/master,1728
JC_Bonnefoy/snt_2019/master,1669
rruizz/inforfis/R,1480
rruizz/inforfis/master,1263
...,...
jibe-b/masse-critique-algo/master,1
jibe-b/mentorat-gender-speech-radio/tinker,1
jibe-b/sabre/master,1
jibe-b/sens-de-la-vie-workflow-brouillon-tests-tuto/dev,1


In [14]:
(df[df.provider == 'Git']
 .spec
 .apply(urllib.parse.unquote, meta=('spec', object))
 .value_counts()
 .to_frame()
 .compute())

,spec
https://git.informatik.uni-hamburg.de/mast/se1/se1-notebook/master,2401
https://gitlab.cs.uni-duesseldorf.de/general/stups/prob-teaching-notebooks/master,2243
https://framagit.org/debimax/cours-debimax/master,1958
https://git.unilim.fr/grossp01/notebooks.git/master,988
https://bitbucket.org/nikiubel/nikiubel.bitbucket.io.git/841046b40e936fa187b974aabb310a6ac0ecd094,794
...,...
https://github.com/rahulrohri/FinalProject/master,1
https://github.com/research-reuse/PISA/master,1
https://github.com/rgoluch/conda/master,1
https://github.com/rmitsch/mybinder-test/51e1202b83cf0270bd5dfa7350d0193b648dc931,1
